In [1]:
import os
import random
import numpy as np
import pandas as pd
import progressbar
import matplotlib.pyplot as plt

import lib
from skimage import io
from skimage.transform import resize
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from importlib import reload
reload(lib)

import warnings
warnings.filterwarnings('ignore')

# Baseline submission


## 1. Get datasets

In [19]:
data_dir = ""

In [3]:
X_train, _,_, y_train, X_test, _,_, y_test, species, _, _, _ = lib.get_splitted_data(data_dir=data_dir, split=1, check_id_sets=True, verbose=1)

The intersection between train and test set is 0
There are 99 classes for the classification task.


In [4]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((990, 195), (990,), (0, 195), (0,))

## 2. ML model

In [5]:
model = LinearDiscriminantAnalysis()

In [6]:
model.fit(X_train, y_train)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

In [7]:
tr_pred = model.predict(X_train)
train_accuracy = (tr_pred == y_train).mean()
train_accuracy

1.0

## 3. Get submission data

In [8]:
X_test, _, _, submission_ids = lib.get_submission_data(data_dir=data_dir)

In [9]:
proba = model.predict_proba(X_test)

In [13]:
if not os.path.exists(data_dir+"/submissions/"):
    os.makedirs(data_dir+"/submissions/")

In [14]:
submission = pd.DataFrame(proba, columns=list(species.keys()))
submission.insert(0, 'id', submission_ids)
submission.reset_index()
submission.to_csv(data_dir+'/submissions/baseline.csv', index=False)
submission.head()

,id,Quercus_Pyrenaica,Salix_Fragilis,Ulmus_Bergmanniana,Rhododendron_x_Russellianum,Castanea_Sativa,Morus_Nigra,Arundinaria_Simonii,Zelkova_Serrata,Quercus_Pontica,...,Ilex_Aquifolium,Lithocarpus_Edulis,Quercus_Infectoria_sub,Tilia_Oliveri,Quercus_Palustris,Quercus_Chrysolepis,Callicarpa_Bodinieri,Acer_Mono,Quercus_Pubescens,Prunus_Avium
0,4,3.631419e-139,8.811537e-268,1.454962e-184,2.151289e-169,6.629601e-214,1.008079e-137,2.022515e-287,1.840142e-163,2.087318e-108,...,5.716791e-110,9.996256e-173,5.255007e-111,4.517866e-125,3.861077e-149,2.311164e-216,3.074608e-175,1.123189e-162,7.775716e-95,7.021876e-271
1,7,6.993240e-84,1.146169e-198,9.911421e-145,2.924359e-218,7.500606e-106,2.232314e-166,2.245440e-212,2.025590e-92,1.394320e-169,...,6.205862e-105,2.612505e-116,2.806150e-67,4.582376e-134,2.795004e-125,2.068862e-147,9.025442e-139,1.871473e-117,8.893069e-170,1.025035e-156
2,9,1.017591e-182,2.744807e-205,7.778647e-112,2.330852e-310,1.074886e-166,3.446930e-127,2.606442e-305,1.214080e-109,9.312262e-236,...,6.197971e-160,3.939491e-232,1.697486e-162,1.435629e-153,4.119382e-233,1.883843e-256,9.368603e-218,1.330126e-168,4.395472e-189,2.646383e-176
3,12,9.753035e-128,7.425222e-182,1.557146e-57,3.307476e-296,1.000000e+00,6.439177e-78,5.548809e-241,8.057444e-52,2.825466e-200,...,9.303643e-161,4.482379e-206,1.182376e-123,6.382430e-159,2.957610e-207,3.798994e-250,3.597696e-179,3.351951e-218,9.667741e-201,2.922576e-37
4,13,8.991551e-232,1.496686e-222,7.712812e-107,0.000000e+00,1.736767e-208,9.236062e-179,0.000000e+00,1.177028e-170,9.360639e-246,...,2.047210e-217,1.729320e-255,3.223095e-183,5.865241e-179,4.111329e-234,2.679754e-296,2.013860e-228,4.027085e-210,2.654275e-247,4.499892e-204


## 4. Send submission
To submit you need [kaggle-api](https://github.com/Kaggle/kaggle-api)

In [36]:
! kaggle competitions submit -c leaf-classification -f {data_dir+"/submissions/baseline.csv"} -m "baseline"

100%|███████████████████████████████████████| 1.26M/1.26M [00:02<00:00, 479kB/s]
Successfully submitted to Leaf Classification

![score](./imgs/baseline_submission.png)